# 🚀 NeuroTrader PPO Training Farm

**Purpose:** Train multiple PPO model variants on GPU for later use in Hybrid AI

**Output:** Models saved to Google Drive → Download to Desktop → Ghost Replay

---

## ⚠️ Before Running:
1. Go to **Runtime → Change runtime type → GPU (T4)**
2. Make sure you have Google Drive space (~500MB per run)

## 1. Setup Environment

In [ ]:
# Check GPU
!nvidia-smi

# Install dependencies
!pip install stable-baselines3[extra] gymnasium pandas pyarrow -q

print("\n✅ Dependencies installed!")

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Create output directory
import os
OUTPUT_DIR = '/content/drive/MyDrive/NeuroTrader_Models'
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"✅ Output directory: {OUTPUT_DIR}")

## 2. Clone NeuroTrader Repository

In [ ]:
# Clone the repo (replace with your repo URL)
!git clone -b NeuroTrader-Windows https://github.com/YOUR_USERNAME/NeuroTrader.git
%cd NeuroTrader

# Or upload data manually if repo is private
print("\n📁 Current directory:", os.getcwd())

## 3. Upload Training Data (If not in repo)

In [ ]:
# Option 1: Upload from local machine
from google.colab import files

print("📤 Upload XAUUSDm_M5_raw.parquet file:")
uploaded = files.upload()

# Move to data folder
import shutil
os.makedirs('data/raw', exist_ok=True)
for filename in uploaded.keys():
    shutil.move(filename, f'data/raw/{filename}')
    print(f"✅ Moved {filename} to data/raw/")

## 4. Define Training Environment

In [ ]:
import gymnasium as gym
import numpy as np
import pandas as pd
from gymnasium import spaces

class SimpleTradingEnv(gym.Env):
    """
    Simplified Trading Environment for PPO Farm.
    Actions: 0=HOLD, 1=BUY, 2=SELL
    """
    def __init__(self, df, lookback=20):
        super().__init__()
        self.df = df.reset_index(drop=True)
        self.lookback = lookback
        self.current_step = lookback
        
        # Actions: HOLD, BUY, SELL
        self.action_space = spaces.Discrete(3)
        
        # Observation: OHLCV + returns + position
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(22,), dtype=np.float32
        )
        
        # State
        self.position = 0  # 0=flat, 1=long
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        
    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.current_step = self.lookback
        self.position = 0
        self.entry_price = 0
        self.balance = 10000
        self.equity = 10000
        return self._get_obs(), {}
    
    def _get_obs(self):
        window = self.df.iloc[self.current_step-self.lookback:self.current_step]
        
        # Compute features
        close = window['close'].values
        returns = np.diff(close) / close[:-1]
        
        # Normalize
        obs = np.concatenate([
            returns[-10:],  # Last 10 returns
            [close[-1] / close[0] - 1],  # Period return
            [(close[-1] - close.min()) / (close.max() - close.min() + 1e-8)],  # Position in range
            [np.std(returns)],  # Volatility
            [self.position],  # Current position
            [self.equity / 10000 - 1],  # PnL %
            np.zeros(7)  # Padding
        ])
        return obs.astype(np.float32)
    
    def step(self, action):
        current_price = self.df.iloc[self.current_step]['close']
        reward = 0
        
        # Execute action
        if action == 1 and self.position == 0:  # BUY
            self.position = 1
            self.entry_price = current_price
            reward = 0.01  # Small entry bonus
            
        elif action == 2 and self.position == 1:  # SELL
            pnl_pct = (current_price - self.entry_price) / self.entry_price
            self.balance *= (1 + pnl_pct)
            reward = pnl_pct * 100  # Scale for learning
            self.position = 0
            self.entry_price = 0
        
        # Update equity
        if self.position == 1:
            unrealized = (current_price - self.entry_price) / self.entry_price
            self.equity = self.balance * (1 + unrealized)
        else:
            self.equity = self.balance
        
        # Move forward
        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        truncated = self.equity < 5000  # Circuit breaker
        
        return self._get_obs(), reward, done, truncated, {'equity': self.equity}

print("✅ Trading Environment defined!")

## 5. PPO Training Farm

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CheckpointCallback
import torch
import json
from datetime import datetime

# Load data
DATA_PATH = 'data/raw/XAUUSDm_M5_raw.parquet'
df = pd.read_parquet(DATA_PATH)
if 'tick_volume' in df.columns:
    df.rename(columns={'tick_volume': 'volume'}, inplace=True)
print(f"✅ Loaded {len(df)} rows from {DATA_PATH}")

# Verify GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"🖥️ Training on: {device.upper()}")

In [ ]:
# Training Configuration
VARIANTS = {
    'aggressive': {'ent_coef': 0.05, 'learning_rate': 5e-4, 'n_steps': 1024},
    'balanced':   {'ent_coef': 0.02, 'learning_rate': 3e-4, 'n_steps': 2048},
    'conservative': {'ent_coef': 0.01, 'learning_rate': 1e-4, 'n_steps': 4096},
}

TOTAL_TIMESTEPS = 500_000  # Per variant
SEEDS = [42, 123, 456]  # Multiple seeds for diversity

print(f"📋 Training Plan:")
print(f"   Variants: {list(VARIANTS.keys())}")
print(f"   Seeds per variant: {len(SEEDS)}")
print(f"   Total models: {len(VARIANTS) * len(SEEDS)}")
print(f"   Steps per model: {TOTAL_TIMESTEPS:,}")

In [ ]:
# Train all variants
trained_models = []

for variant_name, config in VARIANTS.items():
    for seed in SEEDS:
        model_name = f"ppo_{variant_name}_seed{seed}"
        print(f"\n{'='*50}")
        print(f"🚀 Training: {model_name}")
        print(f"   Config: {config}")
        
        # Create env
        env = SimpleTradingEnv(df)
        
        # Create model
        model = PPO(
            "MlpPolicy",
            env,
            device=device,
            verbose=0,
            seed=seed,
            **config
        )
        
        # Train
        start_time = datetime.now()
        model.learn(
            total_timesteps=TOTAL_TIMESTEPS,
            progress_bar=True
        )
        duration = (datetime.now() - start_time).total_seconds()
        
        # Save to Drive
        save_path = f"{OUTPUT_DIR}/{model_name}"
        model.save(save_path)
        
        # Record metadata
        trained_models.append({
            'name': model_name,
            'path': f"{save_path}.zip",
            'variant': variant_name,
            'seed': seed,
            'config': config,
            'timesteps': TOTAL_TIMESTEPS,
            'duration_sec': duration
        })
        
        print(f"✅ Saved: {save_path}.zip ({duration:.1f}s)")

# Save manifest
manifest_path = f"{OUTPUT_DIR}/training_manifest.json"
with open(manifest_path, 'w') as f:
    json.dump(trained_models, f, indent=2)

print(f"\n{'='*50}")
print(f"🎉 TRAINING COMPLETE!")
print(f"   Models saved: {len(trained_models)}")
print(f"   Manifest: {manifest_path}")

## 6. Quick Evaluation

In [ ]:
# Test each model
results = []

for model_info in trained_models:
    model = PPO.load(model_info['path'].replace('.zip', ''), device='cpu')
    env = SimpleTradingEnv(df[-5000:])  # Test on last portion
    
    obs, _ = env.reset()
    total_reward = 0
    trades = 0
    
    for _ in range(4000):
        action, _ = model.predict(obs, deterministic=True)
        obs, reward, done, trunc, info = env.step(action)
        total_reward += reward
        if action in [1, 2]:
            trades += 1
        if done or trunc:
            break
    
    final_return = (info['equity'] / 10000 - 1) * 100
    results.append({
        'model': model_info['name'],
        'return_pct': final_return,
        'trades': trades
    })
    print(f"{model_info['name']}: {final_return:+.2f}% | {trades} trades")

# Save results
results_df = pd.DataFrame(results)
results_df.to_csv(f"{OUTPUT_DIR}/evaluation_results.csv", index=False)
print(f"\n📊 Results saved to {OUTPUT_DIR}/evaluation_results.csv")

## 7. Download Models

ไฟล์ทั้งหมดอยู่ใน Google Drive: `MyDrive/NeuroTrader_Models/`

**วิธีนำไปใช้:**
1. Download โฟลเดอร์ `NeuroTrader_Models` จาก Google Drive
2. Copy ไปยัง Desktop: `NeuroTrader/models/farm/`
3. รัน Ghost Replay: `python scripts/replay_ghosts.py --source models/farm/`

In [ ]:
# List all saved files
print("📁 Files in output directory:")
for f in os.listdir(OUTPUT_DIR):
    size = os.path.getsize(f"{OUTPUT_DIR}/{f}") / 1024 / 1024
    print(f"   {f} ({size:.2f} MB)")